In [ ]:
#after images were cut:
# Remove empty images
import os
import shutil

in_file =  'output/dotasplit/val/rest_out/'
out_file = 'output/dotasplit/val/rest_through/'


for file in os.listdir(in_file+'labelTxt'):
    file_id = file.split('.')[0]
    if os.stat(os.path.join(in_file,'labelTxt',file)).st_size != 0:
        file_path = os.path.abspath(os.path.join(in_file,'images',f'{file_id}.png'))
        file_newpath = os.path.abspath(os.path.join(out_file,'images',f'{file_id}.png'))
        shutil.copyfile(file_path, file_newpath)
        label_path = os.path.abspath(os.path.join(in_file,'labelTxt',f'{file_id}.txt'))
        label_newpath = os.path.abspath(os.path.join(out_file,'labelTxt',f'{file_id}.txt'))
        shutil.copyfile(label_path, label_newpath)


print("Dn")




In [ ]:
# Rename yolo labels dowlnoaded from the Roboflow
import os

dir_in = 'output/dotasplit/val/rest_through/labelTxt/'
dir_img = 'output/dotasplit/val/rest_through/images/'

visited = []
img_visited = []
letters = ['b','c','d','e','f','g','h','i','j','k','l','m','n','o','q','p','r','s','t','u','v','w','x','y','z','alfa','beta','gamma','delta','elipson','zeta','eta','theta','jota','kappa','lambda','mu','nu','xi','omicron','pi','ro','sigma','tau','ypsilon','fi','chi','psi','omega']


def get_new_name(dir, file,name,end,visit):
    for l in letters:
        new_name = name + l
        if new_name in visit:
            continue
        else:
            os.rename(dir + file, dir + new_name + end)
            visit.append(new_name)
            return 0
    return 1

#Labels
end = '.txt'
labels = os.listdir(dir_in)
for file in os.listdir(dir_in):
    if '__' in file:
        name = file.split('__')[0]
    else:
        name = file.split(end)[0]
        visited.append(name)
        continue

    r = get_new_name(dir_in,file,name,end,visited)
    if r == 1:
        print("To few letters")

# Images:
imgend = '.png'
for label in labels:
    file = label.split(end)[0] + imgend
    if '__' in file:
        name = file.split('__')[0]
    else:
        name = file.split(imgend)[0]
        visited.append(name)
        continue


    r = get_new_name(dir_img,file,name,imgend,img_visited)
    if r == 1:
        print("To few letters")

print("Done")


In [ ]:
#(compress images)
# change to jpg

In [ ]:
# Convert Dota dataset to the Coco
import os
from DotaTools.DOTA2COCO import dota2coco

dota_path = 'output/dotasplit/val/done/'
output_path = 'output/dotasplit/val/done/true_labels/'

## Create a new directories
#if not os.path.exists(output_path+'/in'):
#   os.makedirs(output_path+'/in')
#
## Create a new directories
#if not os.path.exists(output_path+'/out'):
#   os.makedirs(output_path+'/out')

dota2coco(srcpath_img=dota_path+'images', srcpath_labels=output_path+'remap_labels',out_json=output_path+'segments_coco.json',end='.png')


In [ ]:
## Script convert Coco to Yolo:
import os
from JSON2YOLO.general_json2yolo import coco_to_yolo
train_coco_path = 'output/dotasplit/val/done/true_labels/segments_coco.json'
output_train_path = 'output/dotasplit/val/done/true_labels/yolo'

# Create a new directories
if not os.path.exists(output_train_path):
   os.makedirs(output_train_path)

coco_to_yolo(train_coco_path,output_train_path)

In [ ]:
## Script convert Yolo to Coco:
import globox
yolo = globox.AnnotationSet.from_yolo_v5(
    folder="output/dotasplit/val/done/combi/yolo",
    image_folder="output/dotasplit/val/done/combi/images"
)
yolo.save_coco(path='output/dotasplit/val/done/combi/coco.json',label_to_id={'0':1,'1':2, '2':3, '3':4, '4':5},imageid_to_id = {key: value for key, value in zip(yolo.image_ids,
 [r for r in range(0,len(yolo.image_ids))])  })

import json
def fix_categories(file,out):
    with open(file,"r") as f:
        data = json.load(f)
    new_data = {}
    new_data['info'] =   {
    "year": "2023",
    "version": "1",
    "description": "Created using globox",
    "contributor": "atm",
    },
    new_data['licenses'] =  [
    {
        "id": 1,
        "url": "https://creativecommons.org/licenses/by/4.0/",
        "name": "CC BY 4.0"
    }
    ]
    new_data['categories'] = [
    {
        "id": 1,
        "name": "airport",
        "supercategory": "objects"
    },
    {
        "id": 2,
        "name": "helicopter",
        "supercategory": "objects"
    },
    {
        "id": 3,
        "name": "oiltank",
        "supercategory": "objects"
    },
    {
        "id": 4,
        "name": "plane",
        "supercategory": "objects"
    },
    {
        "id": 5,
        "name": "warship",
        "supercategory": "objects"
    }
    ]
    new_data['images'] = data['images']
    new_data['annotations'] = data['annotations']
    with open(out,'w') as fout:
        json.dump(new_data, fout , indent=4)

fix_categories('output/dotasplit/val/done/combi/coco.json','output/dotasplit/val/done/combi/coco.json')


In [ ]:
## Script convert Coco to voc:
# create voc dir!
from coco_to_voc import coco2voc
train_coco_path = 'output/dotasplit/val/done/combi/coco.json'
voc_train_path = 'output/dotasplit/val/done/combi/voc'
coco2voc(train_coco_path,voc_train_path,ext='.jpg')


In [ ]:
## Script to convert Coco to Yolified Coco

from yolifycoco import yolifycoco

train_coco_path = 'output/dotasplit/val/done/combi/coco.json'
train_coco_yolo_path = 'output/dotasplit/val/done/combi/cocoyolo.json'
yolifycoco(train_coco_path,train_coco_yolo_path)
